# Mixture model experiment
In this notebook we perform the experiment to verify that we indeed see an exponential amount of gradient queries for stochastic gradient descent with an increasing parameter $d$, whereas we observe a linear relation in the case of the Metropolis-adjusted Langevin algorithm.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import math
import numpy as np
import pandas as pd
import seaborn as sns
from src.mixture import GaussianMixture, random_ball
from tqdm import tqdm
from matplotlib.patches import Circle
import matplotlib.pyplot as plt

## Generating the data
We begin by generating the datasets used in our evaluation. Since we are trying various distributions, we have to make sure that they are all parameterized properly so as to achieve the following properties:
1. The clusters need to be adequately separated.
2. The amount of clusters in our dataset needs to be few ($M=\log_2 d$ in paper).

Something we could consider trying out would be to violate these properties to see what the failure mode is.

In our experiment we use the following distributions (we always let $N=2^d$):
1. Gaussian $\sigma = 1 / \sqrt{d}, M=\log_2 d$
2. Dirichlet
3. Exponential
4. Student's T (included since not log-concave for all parameters)

For our experiment to work, we need to generate new such problems for an increasing parameter $d$. Therefore we will need in total $4d$ datasets to work with.

## Experiment
Now we begin the experiment. The setup is quite simplistic:
1. Iterate over parameter $d$.
2. Gather our distribution datasets for that parameter $d$.
3. Estimate the parameters of the distribution using expectation-maximization.
4. Estimate the parameters of the distribution using MALA.
5. Save the amount of gradient queries required for both approaches.
6. Create line plot of required gradient queries for convergence.

In [ ]:
def plot_points_in_ball(num_points: int = 1000, radius: int = 1, dim: int = 2):
    if dim < 2 or dim > 3:
        raise ValueError('Invalid dimension')

    points = random_ball(num_points, dim, radius=radius)

    subplot_kw = {}

    if dim == 3:
        subplot_kw=dict(projection='3d')
        sns.set(style = "darkgrid")

    fig, ax = plt.subplots(subplot_kw=subplot_kw)

    if dim == 2:
        ax.set_aspect('equal')
        patch = Circle((0, 0), radius, fill=False, ls='-', lw=0.25)
        ax.add_patch(patch)

    ax.scatter(*np.split(points, dim, axis=1), marker='.')
    plt.show()

In [ ]:
%matplotlib notebook
plot_points_in_ball(dim=2)

In [ ]:
m = GaussianMixture(5)

In [ ]:
m.objective()

In [ ]:
dims = range(2, 10)

models = {
    m.__name__: m
    for m in [
        GaussianMixture
    ]
}

results_df = pd.DataFrame(
    np.nan,
    index=dims,
    columns=pd.MultiIndex.from_product((models.keys(), ['em', 'mala']))
)

In [ ]:
results_df

In [ ]:
for d in tqdm(dims):
    for name, constructor in models.items():
        model = constructor(d)
        
        # Run EM
        results_df.loc[d, (name, 'em')] = em(model)
        
        # Run MALA
        results_df.loc[d, (name, 'mala')] = mala(model)

## Analysis
Now that we have all the necessary data we can examine our results to see if they make sense in the context of the paper we are referencing.

We begin by generating figures for the individual mixture problems:

In [ ]:
# Gaussian
gdf = pd.DataFrame({'Dimensions': ds + ds, 
                    'Gradient queries': np.concatenate((gauss_gq_EM, gauss_gq_MALA)), 
                    'Algorithm': ['EM']*iterations + ['MALA']*iterations})
sns.lineplot(data=gdf, x='Dimensions', y='Gradient queries', hue='Algorithm').set_title('Gaussian Mixture Model')
plt.show()

In [ ]:
# Dirichlet
ddf = pd.DataFrame({'Dimensions': ds + ds, 
                    'Gradient queries': np.concatenate((dirichlet_gq_EM, dirichlet_gq_MALA)), 
                    'Algorithm': ['EM']*iterations + ['MALA']*iterations})
sns.lineplot(data=ddf, x='Dimensions', y='Gradient queries', hue='Algorithm').set_title('Dirichlet Mixture Model')
plt.show()

In [ ]:
# Exponential
edf = pd.DataFrame({'Dimensions': ds + ds, 
                    'Gradient queries': np.concatenate((exponential_gq_EM, exponential_gq_MALA)), 
                    'Algorithm': ['EM']*iterations + ['MALA']*iterations})
sns.lineplot(data=edf, x='Dimensions', y='Gradient queries', hue='Algorithm').set_title('Exponential Mixture Model')
plt.show()

In [ ]:
# Student's T
tdf = pd.DataFrame({'Dimensions': ds + ds, 
                    'Gradient queries': np.concatenate((students_gq_EM, students_gq_MALA)), 
                    'Algorithm': ['EM']*iterations + ['MALA']*iterations})
sns.lineplot(data=tdf, x='Dimensions', y='Gradient queries', hue='Algorithm').set_title('Student\'s T Mixture Model')
plt.show()

BLABLABLA analysis BLABLABLA

Let us now also generate a large figure containing the data for all datasets:

In [ ]:
adf = pd.DataFrame({'Dimensions': ds*8, 
                    'Gradient queries': np.concatenate((gaussian_gq_EM, gaussian_gq_MALA,
                                                        dirichlet_gq_EM, dirichlet_gq_MALA,
                                                        exponential_gq_EM, exponential_gq_MALA, 
                                                        students_gq_EM, students_gq_MALA)), 
                    'Combination': ['Gaussian (EM)']*iterations + ['Gaussian (MALA)']*iterations +
                                   ['Dirichlet (EM)']*iterations + ['Dirichlet (MALA)']*iterations +
                                   ['Exponential (EM)']*iterations + ['Exponential (MALA)']*iterations +
                                   ['Student\'s T (EM)']*iterations + ['Student\'s T (MALA)']*iterations +})
sns.lineplot(data=adf, x='Dimensions', y='Gradient queries', hue='Combination').set_title('All Mixture Models')
plt.show()